<a href="https://colab.research.google.com/github/Elwing-Chou/ml0804/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)
print("extract on:", dataset)

In [ ]:
import glob
import pandas as pd
import os
def getdata(dataset, t):
    dn = os.path.dirname(dataset)
    dn = os.path.join(dn, "aclImdb", t)
    pos = glob.glob(os.path.join(dn, "pos", "*.txt")) + glob.glob(os.path.join(dn, "pos", "*.TXT"))
    neg = glob.glob(os.path.join(dn, "neg", "*.txt")) + glob.glob(os.path.join(dn, "neg", "*.TXT"))
    contents = []
    for fn in pos + neg:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(pos) + [0] * len(neg)
    })
    return df
train_df = getdata(dataset, "train")
test_df = getdata(dataset, "test")
train_df

In [ ]:
test_df